<a href="https://colab.research.google.com/github/gabrielursulino/ciencia-de-dados-e-analytics/blob/main/mvp_machine_learning_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP: Machine Learning & Analytics
**Nome:** Gabriel Lopes Ursulino

**Matrícula:** 4052025000406

**Dataset original:** [Cerebral Stroke Prediction-Imbalanced Dataset](https://www.kaggle.com/datasets/shashwatwork/cerebral-stroke-predictionimbalaced-dataset/data)

# Descrição do Problema


O Acidente Vascular Cerebral (AVC), também conhecido pelo termo em inglês *stroke*, é uma das principais causas de morte e incapacidade no mundo. O AVC acontece quando vasos que levam sangue ao cérebro entopem ou se rompem, provocando a paralisia da área cerebral que ficou sem circulação sanguínea. O AVC pode ser hemorrágico, quando há rompimento de um vaso cerebral, provocando hemorragia, ou isquêmico, quando há obstrução de uma artéria, impedindo a passagem de oxigênio para células cerebrais, que acabam morrendo. Essa obstrução pode acontecer devido a um trombo (trombose) ou a um êmbolo (embolia).

O conjunto de dados *Cerebral Stroke Prediction-Imbalanced Dataset* é um conjunto de dados multivariado que reúne diversas características que podem estar associadas ao risco de AVC, como idade, hábitos, estilo de vida e condições prévias de saúde como hipertensão e doenças cardiovasculares.

O objetivo deste MVP é dar continuidade no trabalho desenvolvido na Sprint anterior e desenvolver um modelo de Machine Learning capaz de classificar os pacientes com risco de sofrer um AVC.

## Hipóteses do Problema

É possível, a partir dos dados extremamente desbalanceados e com escassez de variáveis de entrada, construir um modelo modelo de classificação robusto e confiável para a detecção de ocorrência de AVCs afim de fornecer insumos para estratégias de prevenção?

As técnicas de pré-processamento e balanceamento são capazes de oferecer melhorias significativas quando comparados a um modelo simples sem qualquer técnica aplicada?

## Tipo de Problema

Este é um problema de classificação supervisionada. Dado um conjunto de características (idade, genero, hipertenso, fumante etc.), o objetivo é avaliar os fatores de risco que contribuem para ocorrência de AVC para prever futuras ocorrências.

## Seleção de Dados

O dataset original é amplamente disponível. Não é necessária uma etapa de seleção de dados externa, pois o dataset já está curado e pronto para uso. No entanto, o dataset apresentado é uma versão que já passou por uma análise exploratória e algumas etapas de pré-processamento.

## Atributos do Dataset


O dataset contém 43.389 amostras, com os seguintes atributos:

- **`gender`** (Gênero: "Male", "Female" or "Other"), em português: "masculino", "feminino" ou "outro", respectivamente.
- **`age`** (Idade do paciente em anos)
- **`hypertension`** (hipertensão: 0 se o paciente não possui, 1 se o paciente possui)
- **`heart_disease`** (doença cardiovascular: 0 se o pacitente não possui, 1 se o paciente possui)
- **`ever_married`** (Se o paciente já foi casado: "Yes" or "No") em português : "sim" ou "não", respectivamente.
- **`Residence_type`** (Local de moradia: "Rural" or "Urban") em português: "rural" ou "urbano", respectivamente.
- **`avg_glucose_level`** (Nível médio de glicose no sangue do paciente em mg/dL)
- **`bmi`** (Body Mass Index, ou Índice de Massa Corporal (IMC))
- **`stroke`** (AVC: 0 se o paciente não teve AVC, 1 se o paciente teve)
- **`work_type_Never_worked`**, **`work_type_Private`**, **`work_type_Self-employed`**, **`work_type_children`**  (Colunas geradas a partir do One-Hot Encoding da coluna **`work_type`** do dataset original. Informa se o tipo de trabalho é do setor privado, autônomo, se nunca trabalhou etc.)
- **`smoking_status_never smoked`**, **`smoking_status_smokes`**, **`smoking_status_unknown`** (Colunas geradas a partir do One-Hot Encoding da coluna **`smoking_status`** do dataset original. Informa o status de tabagismo, **`never smoked`** = nunca fumou, **`formerly smoked`** = ex-fumante, **`smokes`** = fumante)

# Importação das Bibliotecas Necessárias e Carga de Dados

In [ ]:
# Importação das bibliotecas e reprodutibilidade
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier # Ensembles
from sklearn.metrics import accuracy_score, precision_recall_curve, make_scorer, recall_score, precision_score, classification_report, roc_auc_score, confusion_matrix # Para a exibição das métricas
from sklearn.neighbors import KNeighborsClassifier # algoritmo KNN
from sklearn.tree import DecisionTreeClassifier # algoritmo Árvore de Classificação
from sklearn.naive_bayes import GaussianNB # algoritmo Naive Bayes
from sklearn.svm import SVC # algoritmo SVM
from sklearn.linear_model import LogisticRegression #algoritmo Regressão Logística
from sklearn.datasets import make_classification
from tensorflow import keras
from sklearn.utils import class_weight
import time # Para medição do tempo de treino


# Reprodutibilidade
SEED = 42

In [ ]:
# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/gabrielursulino/ciencia-de-dados-e-analytics/refs/heads/main/datasets/stroke_dataset_preproc.csv"

# Lê o arquivo
avcpreproc = pd.read_csv(url, delimiter=',')

# Exibe as primeiras linhas do novo DataFrame
avcpreproc.head()

*Para mais detalhes a respeito do dataset original, acesse:* https://www.kaggle.com/datasets/shashwatwork/cerebral-stroke-predictionimbalaced-dataset/data

#Informações Gerais e Resumo da AED e Pré-processamento (realizados na Sprint anterior)

Na Sprint anterior, durante a etapa de Análise Exploratória foi identificada a necessidade de tratamento dos dados ausentes nas colunas ***bmi*** e ***smoking_status***.

A coluna ***bmi*** apresentou valores nulos em aproximadamente 3.37% dos registros e a análise dos boxplots revelou a presença de outliers. A estratégia de imputação escolhida foi preencher estes valores ausentes com a mediana, por ser uma medida mais robusta a valores extremos e, portanto, a mais indicada para não distorcer a distribuição original dos dados.

A coluna ***smoking_status*** apresentou uma quantidade massiva de linhas vazias, correspondendo a mais de 30% do dataset. Diante de um volume tão grande, a imputação com a moda foi descartada por introduzir um viés significativo nos dados.  A exclusão das linhas acarretaria em uma grande perda de informação e a exclusão da coluna também não foi considerada, visto que identificamos o status de tabagismo como um fator de risco para ocorrência de AVC.

Logo, a decisão estratégica foi imputar os valores vazios desta coluna com uma nova categoria, ***'unknown'***. Essa abordagem tem a dupla vantagem de preservar todas as instâncias do dataset e, ao mesmo tempo, transformar a falta de informação em uma feature que o próprio modelo de Machine Learning poderá usar para determinar se a ausência de dados sobre tabagismo é, por si só, um fator preditivo.

Embora tenha sido localizada a presença de diversos outliers, principalmente em ***avg_glucose_level*** (Nível de Glicose) e ***bmi*** (IMC), não se tratam de erros de digitação ou medição. São medidas válidas e extremamente valiosas para o modelo de predição. Por essa razão, a decisão estratégica é não fazer nenhum tipo de tratamento e manter os dados originais.

Ainda na etapa de pré-processamento, foi realizada a conversão das variáveis categóricas ***gender***, ***ever_married*** e ***Residence_type*** em valores binários e utilizei One-Hot Encoding para tratamento das colunas ***work_type*** e ***smoking_status***.

Em resumo, após extensa análise e tratamento, o dataset está pronto que o modelo seja trabalhado.

*Para mais detalhes a respeito do trabalho que foi desenvolvido, clique no link a seguir:* https://github.com/gabrielursulino/ciencia-de-dados-e-analytics/blob/main/MVP/mvp_analise_de_dados_e_boas_praticas.ipynb

In [ ]:
# Verificando as dimensões
print(f"Dimensões do Dataset: {avcpreproc.shape}")
print("-" * 30)

# Verificando os tipos de dados e valores não-nulos
print("Informações Gerais e Tipos de Dados:")
avcpreproc.info()

O dataset que será trabalhado possui 43.389 instâncias e 16 colunas. Temos 3 variáveis do tipo *float* e 13 do tipo *int*.

In [ ]:
# Informação em % da distribuição
print("Proporção das classes em 'stroke' em %:")
print(avcpreproc['stroke'].value_counts(normalize=True) * 100)

# Gráfico de barras para análise da variável alvo
plt.figure(figsize=(7, 5))
sns.countplot(x='stroke', hue='stroke', data=avcpreproc, palette={0: '#ADD8E6', 1: '#FF0000'}, legend=False)
plt.title('Distribuição da ocorrência de AVC')
plt.xlabel('')
plt.ylabel('Contagem')
plt.xticks(ticks=[0, 1], labels=['Não teve AVC (0)', 'Teve AVC (1)'])
plt.show()

Com esse gráfico podemos confirmar que se trata de um dataset extremamente desbalanceado em termos de classes (98,20% contra 1,80%).  Isso exigirá balanceamento para evitar que o modelo seja enviesado para a classe majoritária.

# Preparação dos dados

##Separação entre Treino e Teste

No código abaixo optei em dividir o conjunto entre 70/30, garantindo ao modelo um volume de dados robusto para tentar aprender os complexos padrões subjacentes aos fatores de risco de um AVC.

In [ ]:
# Separação dos Dados (Features e Target)
X = avcpreproc.drop('stroke', axis=1)
y = avcpreproc['stroke']

# Divisão estratificada em conjuntos de treino e teste para manter a proporção
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.3, random_state=SEED, stratify = y)

print('Tamanho das separações:')
print(f"Treino: {X_train.shape}, {y_train.shape}")
print(f"Teste: {X_test.shape}, {y_test.shape}")
print(f"Proporção de stroke no treino: {y_train.mean():.4f}")
print(f"Proporção de stroke no teste: {y_test.mean():.4f}")

##Padronização

Devido à existência de outliers e às distribuições assimétricas, optei pela padronização. A normalização comprime as distribuições e um único outlier poderia achatar a escala dos dados restantes. Com a padronização eu evito esse problema, pois ela é mais robusta aos outliers ao utilizar média e desvio padrão para cálculo, ao invés de mínimo e máximo.

A partir dos histrogramas abaixo, podemos verificar a distribuição dos dados no dataset.

In [ ]:
# Confirguração e título
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plt.suptitle('Histograma das Variáveis Contínuas', fontsize=16)

# Histograma 1: age (idade)
sns.histplot(ax=axes[0], data=avcpreproc, x='age', kde=True, color='lightgreen')
axes[0].set_ylabel('Frequência')
axes[0].set_xlabel('Distribuição de Idade')

# Histograma 2: avg_glucose_level (nível de glicose)
sns.histplot(ax=axes[1], data=avcpreproc, x='avg_glucose_level', kde=True, color='lightblue')
axes[1].set_ylabel('Frequência')
axes[1].set_xlabel('Nível Médio de Glicose')

# Histograma 3: bmi (IMC)
sns.histplot(ax=axes[2], data=avcpreproc, x='bmi', kde=True, color='red')
axes[2].set_ylabel('Frequência')
axes[2].set_xlabel('IMC')

plt.show()

Abaixo será feita a padronização dos dados dessas colunas utilizando o ColumnTransformer para posterior adição aos pipelines dos testes.

In [ ]:
# Define quais colunas serão padronizadas
vcontinuas = ['age', 'avg_glucose_level', 'bmi']

# Pipeline de pré-processamento focado na padronização
preprocessor = ColumnTransformer(transformers=[('scaler_continuas', StandardScaler(), vcontinuas)],remainder='passthrough')

# Aplica o pré-processador
X_train_processed = preprocessor.fit_transform(X_train)

# Separa as colunas que não foram padronizadas
passthrough_cols = [col for col in X_train.columns if col not in vcontinuas]
processed_cols = vcontinuas + passthrough_cols

# Converte e define dataframe que será usado como conjunto de treino
X_train_scaled = pd.DataFrame(X_train_processed, columns=processed_cols)

In [ ]:
#Exibe as primeiras linhas dos dados de treino padronizados
X_train_scaled.head()

In [ ]:
# Confirguração e título dos histogramas padronizados
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plt.suptitle('Histograma das Variáveis Contínuas Padronizadas', fontsize=16)

# Histograma padronizado 1: age (idade)
sns.histplot(ax=axes[0], data=X_train_scaled, x='age', kde=True, color='lightgreen')
axes[0].set_ylabel('Frequência')
axes[0].set_xlabel('Distribuição de Idade')

# Histograma padronizado 2: avg_glucose_level (nível de glicose)
sns.histplot(ax=axes[1], data=X_train_scaled, x='avg_glucose_level', kde=True, color='lightblue')
axes[1].set_ylabel('Frequência')
axes[1].set_xlabel('Nível Médio de Glicose')

# Histograma 3: bmi (IMC)
sns.histplot(ax=axes[2], data=X_train_scaled, x='bmi', kde=True, color='red')
axes[2].set_ylabel('Frequência')
axes[2].set_xlabel('IMC')

plt.show()

Os histogramas mostram que a forma geral da distribuição foi preservada e os valores foram transformados para ter uma média próxima de zero e um desvio padrão de um, o que era esperado após a padronização.

In [ ]:
# Aplica a transformação no conjunto de teste
X_test_processed = preprocessor.transform(X_test)

# Converte e define dataframe que será usado como conjunto de teste
X_test_scaled = pd.DataFrame(X_test_processed, columns=processed_cols)

# Exibe conjunto de teste
X_test_scaled.head()

## Baseline

Optei por utilizar Regressão Logística como baseline por sua simplicidade e alta interpretabilidade, estabelecendo um benchmark de performance e complexidade. Ela define um "piso de performance" claro, que qualquer modelo mais sofisticado deve obrigatoriamente superar para justificar sua utilização.

In [ ]:
# Baseline - Regressão Logística
start_time = time.time() # inicia o cronômetro

# Define a Baseline como Regressão Logística
lr_baseline = LogisticRegression(random_state=SEED, max_iter=1000)
lr_baseline.fit(X_train_scaled, y_train) # Treina a baseline no conjunto de treino

training_time = time.time() - start_time # finaliza e calcula o tempo

# Previsões
y_pred_baseline = lr_baseline.predict(X_test_scaled)
y_proba_baseline = lr_baseline.predict_proba(X_test_scaled)[:, 1]

# Métricas
baseline_time = time.time() - start_time
print(f"Tempo de treino (baseline): {baseline_time:.2f} segundos")
print("\nRelatório de classificação (Baseline - Regressão Logística):")
print(classification_report(y_test, y_pred_baseline, zero_division=0))
print(f"AUC-ROC: {roc_auc_score(y_test, y_proba_baseline):.4f}")

A criação de uma baseline, treinada propositalmente nos dados de treino originais e desbalanceados serve como um benchmark mínimo e realista para o projeto. O resultado para a classe minoritária é, na verdade, um diagnóstico poderoso: ele confirma a gravidade do desbalanceamento, expõe a inutilidade de um modelo ingênuo que apenas aprende a prever o resultado mais comum, e estabelece que qualquer modelo só terá valor real se superar significativamente essa performance, principalmente ao alcançar um recall positivo para os casos de AVC. Usar essa abordagem, em vez de uma baseline artificialmente otimizada com dados já balanceados, permite mensurar o verdadeiro valor agregado pelas técnicas de modelagem e balanceamento subsequentes, garantindo que qualquer melhoria seja genuína e não uma ilusão criada por uma referência fraca.

##Balanceamento

A análise exploratória revelou um severo desbalanceamento de classes no dataset, onde a classe minoritária (stroke=1) representa apenas 1,8% do total de registros. Diante deste cenário, é necessário utilizar alguma técnica de balanceamento para evitar um modelo com acurácia falsamente alta prevendo somente a não ocorrência de AVC para todos os pacientes. Optei por utilizar as técnicas SMOTE, UnderSampling, e SMOTEENN e avaliar como os modelos se comportam em cada uma delas. Abaixo é possível verificar a distribuição das clases após a aplicação de cada uma das técnicas aplicadas.

###SMOTE

O **SMOTE** cria novos dados sintéticos, enriquecendo o dataset e ajudando o modelo a criar uma região de decisão mais ampla e generalista para a classe minoritária, reduzindo o risco de overfitting.

In [ ]:
# Balanceamento de Classes usando SMOTE no conjunto de treino somente
print("Distribuição de classes antes do SMOTE:", y_train.value_counts())

smote = SMOTE(random_state=SEED)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print("\nDistribuição de classes depois do SMOTE:", y_train_smote.value_counts())

Acima podemos ver que agora as classes estão balanceadas no conjunto de treino.

###UnderSampling

O **UnderSampling** tem a capacidade de balancear o dataset reduzindo o número de exemplos da classe majoritária, o que mitiga o viés do modelo em favor dessa classe e pode diminuir drasticamente o custo computacional e o tempo de treinamento. Por outro lado, sua maior desvantagem é o risco significativo de descartar informações importantes e úteis ao remover dados da classe majoritária, o que pode levar o modelo a não aprender os padrões de forma completa e, consequentemente, resultar em um desempenho de generalização inferior em dados reais.

In [ ]:
# Balanceamento de Classes usando UnderSampling no conjunto de treino somente
print("Distribuição de classes antes do UnderSampling:", y_train.value_counts())

under_sampler = RandomUnderSampler(random_state=SEED)
X_train_undersampled, y_train_undersampled = under_sampler.fit_resample(X_train_scaled, y_train)

print("\nDistribuição de classes depois do UnderSampling:", y_train_undersampled.value_counts())

Acima podemos ver que agora as classes estão balanceadas no conjunto de treino.

###SMOTEENN

Essa técnica primeiro aplica o SMOTE para criar novas amostras e depois faz undersampling para remover amostras ruidosas ou ambíguas, "limpando" a fronteira de decisão.

In [ ]:
print("Distribuição de classes antes do SMOTEENN:", y_train.value_counts())

# Balanceamento de Classes usando SMOTEENN no conjunto de treino somente
sampler = SMOTEENN(random_state=SEED)
X_train_smoteenn, y_train_smoteenn = sampler.fit_resample(X_train_scaled, y_train)

print("\nDistribuição de classes depois do SMOTEENN:", y_train_smoteenn.value_counts())

Acima podemos ver que agora as classes estão balanceadas no conjunto de treino.

# Modelagem

Abaixo será executada uma série de testes, utilizando os modelos candidatos com o dataset original e com técnicas de balanceamento. Optei por comentar o algoritmo SVM, pois o tempo de processamento aumenta drasticamente e o resultado não é interessante.

Na primeira célula com os modelos candidatos, os modelos serão treinados sem validação cruzada. Já na segunda, o código aplica a validação cruzada com 5 folds.

As métricas mais relevantes para esse projeto são a precisão, o recall e a f1-score. Isso se deve à importancia do modelo acertar os casos positivos de AVC alarmando o mínimo de pessoas saudáveis o possível. Ou seja, preciso do máximo de verdadeiros positivos com o mínimo de falsos positivos.

In [ ]:
# 1. Dicionário com os modelos
models = {
    'Logistic Regression': LogisticRegression(random_state=SEED, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=SEED),
    'Gradient Boosting': GradientBoostingClassifier(random_state=SEED),
    'NB': GaussianNB(),
    'CART': DecisionTreeClassifier(),
    #'SVM': SVC(random_state=SEED, probability=True), # Devido ao tempo de treinamento elevado e o resultado ruim, optei por comentar esse algoritmo
    'K-Nearest Neighbors': KNeighborsClassifier()
}

# 2. Dicionário com as estratégias de balanceamento
balancers = {
    'Original': None, # Sem balanceamento
    'SMOTE': SMOTE(random_state=SEED),
    'UnderSample': RandomUnderSampler(random_state=SEED),
    'SMOTEENN': SMOTEENN(random_state=SEED)
}

# 3. Definição do passo de pré-processamento
vcontinuas = ['age', 'avg_glucose_level', 'bmi']
preprocessor = ColumnTransformer(transformers=[('scaler_continuas', StandardScaler(), vcontinuas)],remainder='passthrough')

# 4. Lista para armazenar todos os resultados
results_list = []

# ----------- Loop de experimentação com Pipeline ---------

for bal_name, balancer in balancers.items():
    print(f"\n{'='*20} Processando Estratégia: {bal_name.upper()} {'='*20}")

    # Loop para treinar e avaliar cada modelo com a estratégia atual
    for model_name, model in models.items():
        print(f"--- Treinando {model_name} ---")

        # Cria um pipeline completo para cada combinação
        steps = [('preprocessor', preprocessor)]
        if balancer:
            steps.append(('sampler', balancer))
        steps.append(('model', model))

        pipeline = ImbPipeline(steps=steps)

        # Inicia o cronômetro
        start_time = time.time()

        # Treina o modelo com pipeline nos dados
        pipeline.fit(X_train, y_train)

        # Finaliza e calcula o tempo
        training_time = time.time() - start_time

        # Faz previsões no conjunto de Teste
        y_pred = pipeline.predict(X_test)

        # Obtem probabilidades para o AUC-ROC. Trata exceções para modelos que não têm predict_proba.
        try:
            y_proba = pipeline.predict_proba(X_test)[:, 1]
        except AttributeError:
            y_proba = y_pred # Para modelos como SVM sem probability=True

        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

        results_list.append({
            'Estrategia': bal_name,
            'Modelo': model_name,
            'Precisao (AVC)': report.get('1', {}).get('precision', 0),
            'Recall (AVC)':   report.get('1', {}).get('recall', 0),
            'F1-Score (AVC)': report.get('1', {}).get('f1-score', 0),
            'AUC-ROC':        roc_auc_score(y_test, y_proba),
            'Tempo (s)':      training_time
        })

        print(f"Tempo de Treino: {training_time:.2f} segundos")


# --- Visualização dos resultados ---

# Cria o DataFrame final com todos os resultados
results_df = pd.DataFrame(results_list)

# Ordena o DataFrame para ver facilmente os melhores resultados
sorted_results = results_df.sort_values(by=['Precisao (AVC)','Recall (AVC)'], ascending=False)

# Exibe os resultados
print("\n\n" + "="*50)
print("TABELA DE COMPARAÇÃO DE MODELOS")
print("="*50)
print(sorted_results.round(2)) # Arredonda para 2 casas decimais

In [ ]:
# 1. Dicionário com os modelos
#models = {
    #'Logistic Regression': LogisticRegression(random_state=SEED, max_iter=1000),
    #'Random Forest': RandomForestClassifier(random_state=SEED),
    #'Gradient Boosting': GradientBoostingClassifier(random_state=SEED),
    #'NB': GaussianNB(),
    #'CART': DecisionTreeClassifier(),
    #'SVM': SVC(random_state=SEED, probability=True), # Devido ao tempo de treinamento elevado e o resultado ruim, optei por comentar esse algoritmo
    #'K-Nearest Neighbors': KNeighborsClassifier()
#}

# 2. Dicionário com as estratégias de balanceamento
#balancers = {
    #'Original': None, # Sem balanceamento
    #'SMOTE': SMOTE(random_state=SEED),
    #'UnderSample': RandomUnderSampler(random_state=SEED),
    #'SMOTEENN': SMOTEENN(random_state=SEED)
#}

# 3. Lista para armazenar todos os resultados
#results_list = []

# Usa 5 folds estratificados para processamento mais rápido
#cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# ----------- Loop de experimentação ---------
#for bal_name, balancer in balancers.items():
    #print(f"\n{'='*20} Processando Estratégia: {bal_name.upper()} {'='*20}")

    #for model_name, model in models.items():
        #print(f"--- Avaliando {model_name} com Validação Cruzada ---")

        # # Inicia o cronômetro
        #start_time = time.time()

        # Cria um pipeline completo para cada combinação
        #steps = [('preprocessor', preprocessor)]
        #if balancer:
            #steps.append(('sampler', balancer))
        #steps.append(('model', model))

        #pipeline = ImbPipeline(steps=steps)

        # Calcula os scores de Recall
        #recall_cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv_strategy, scoring=make_scorer(recall_score, pos_label=1), n_jobs=-1)

        # Calcula os scores de Precisão
        #precision_cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv_strategy, scoring=make_scorer(precision_score, pos_label=1, zero_division=0), n_jobs=-1)

        # Finaliza e calcula o tempo
        #training_time = time.time() - start_time

        # Salva a MÉDIA dos resultados da validação cruzada
        #results_list.append({
            #'Estrategia': bal_name,
            #'Modelo': model_name,
            #'Recall Médio (CV)': recall_cv_scores.mean(),
            #'Precisao Média (CV)': precision_cv_scores.mean(),
            #'Tempo (s)': training_time
        #})

        #print(f"Tempo de Treino: {training_time:.2f} segundos")

# --- Visualização dos resultados ---

# Cria o DataFrame final com todos os resultados
#results_df = pd.DataFrame(results_list)
# Renomeia a variável para evitar conflito com o DataFrame original
#sorted_results_cv = results_df.sort_values(by=['Precisao Média (CV)','Recall Médio (CV)'], ascending=False)

# Exibe os resultados
#print("\n\n" + "="*50)
#print("TABELA DE COMPARAÇÃO DE MODELOS (Validação Cruzada)")
#print("="*50)
#print(sorted_results_cv.round(2)) # Arredonda para 2 casas decimais

Na célula acima é executado o treinamento dos modelos candidatos com e sem as técnicas de balanceamento e utilizando a validação cruzada com 5 folds. Devido ao tempo de execução, próximo aos 10 minutos, optei por deixar o código comentado e abaixo uma tabela resumo com os modelos que apresentaram melhor resultado. Podemos ver mais uma vez que os resultados sofreram pouca alteração, tendo no geral uma precisão pior.

---

| Estrategia | Modelo | Recall Médio (CV) | Precisao Média (CV) | Tempo de treinamento (s) |
| :--- | :--- | :--- | :--- | :--- |
| Original | NB | 1.00 | 0.02 | 0.54 |
| UnderSample | NB | 1.00 | 0.02 | 0.50 |
| SMOTEENN | NB | 1.00 | 0.02 | 38.55 |
| SMOTE | NB | 1.00 | 0.02 | 0.73 |
| SMOTEENN | Logistic Regression | 0.82 | 0.05 | 39.01 |
| UnderSample | Random Forest | 0.82 | 0.05 | 2.82 |
| UnderSample | Gradient Boosting | 0.81 | 0.05 | 2.48 |
| SMOTE | Logistic Regression | 0.81 | 0.05 | 2.46 |

De forma geral, utilizar a validação cruzada não fez sentido no meu dataset, pois além do tempo de treinamento ser muito maior, o resultado foi inferior.

Os testes mostraram uma precisão baixíssima, o que indica que os modelos estão causando um alto número de falsos positivos. No contexto do meu problema, isso é ruim porque cria um alarme desnecessário em muitas pessoas que não terão AVC.

No entanto, o recall é interessante, indicando que o modelo não é cego e consegue identificar os verdadeiros positivos de forma minimamente satisfatória.

---

Após o teste e compração de diversos modelos e utilizando diversas técnicas, a Regressão Logística com SMOTE se apresentou como mais promissora por aprensentar um melhor equilíbrio entre recall e precisão, com 0.06 e 0.81 respectivamente.

Essa análise inicial indicou modelos com precisão muito abaixo da aceitável, tornando o modelo quase inútil até mesmo para ser usado como uma triagem.

**Tentativa com Class Weigth no melhor modelo (Regressão Logística)**

In [ ]:
# ---- Avaliação com Pipeline ----

# Componente A: O Pré-processador
# Definido na etapa de padronização

# Componente B: Modelo base que dará muito mais importância à classe minoritária
lr_com_peso = LogisticRegression(class_weight='balanced',random_state=SEED)

# --- Montagem do Pipeline ---
pipeline_lr_weight = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', lr_com_peso)
])

# Treinamento do modelo
pipeline_lr_weight.fit(X_train, y_train)

# Avalia os resultados
y_pred = pipeline_lr_weight.predict(X_test)
print(classification_report(y_test, y_pred))

Aqui tentei utilizar o modelo com melhor resultado no dataset sem balanceamento, mas focando em dar mais peso para a classe desbalanceada. No entanto o resultado do recall foi um pouco pior, então desconsiderarei.

**Tentativa com ensembles Bagging Classifier e AdaBoostClassifier**

In [ ]:
# ---- Avaliação com Pipeline ----

# Componente A: O Pré-processador
# Definido na etapa de padronização

# Componente B: O Balanceador
sampler = SMOTE(random_state=SEED) # Melhor balanceador encontrado para esse teste

# Componente C: Modelo base de Regressão Logística
log_reg = LogisticRegression(max_iter=1000)

# Cria o ensemble Bagging com a Regressão Logística como estimador base
bagging_model = BaggingClassifier(estimator=log_reg, n_estimators=50, random_state=SEED)

# --- Montagem do Pipeline ---
pipeline_bagging_model = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampler', sampler),
    ('model', bagging_model)
])

# Treina o modelo ensemble
start_time = time.time() # Inicia o cronômetro
pipeline_bagging_model.fit(X_train, y_train)
training_time = time.time() - start_time
print(f"Tempo de Treino: {training_time:.2f} segundos")

# Faz previsões
y_pred_bagging = pipeline_bagging_model.predict(X_test)

# Avaliação
print(classification_report(y_test, y_pred_bagging))

Acima podemos ver o resultado identico ao modelo de regressão logística mais simples, mas consumindo um tempo de treino muito maior, o que não justifica a utilização desse modelo mais complexo.

In [ ]:
# ---- Avaliação com Pipeline ----

# Componente A: O Pré-processador
# Definido na etapa de padronização

# Componente B: O Balanceador
sampler = SMOTEENN(random_state=SEED) # Melhor balanceador encontrado para esse teste

# Componente C: ensemble AdaBoost com a Regressão Logística como estimador base
adaboost_model = AdaBoostClassifier(estimator=LogisticRegression(max_iter=1000), n_estimators=50, random_state=SEED)

# --- Montagem do Pipeline ---
pipeline_adaboost = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampler', sampler),
    ('model', adaboost_model)
])

# Treina o modelo
start_time = time.time() # Inicia o cronômetro
pipeline_adaboost.fit(X_train, y_train)
training_time = time.time() - start_time
print(f"Tempo de Treino: {training_time:.2f} segundos")

# Faz previsões
y_pred_boost = pipeline_adaboost.predict(X_test)

# Avaliação
print(classification_report(y_test, y_pred_boost))

Utizando o AdaBoost o recall melhorou um pouco, mas a precisão caiu. O valor da precisão já está crítico, então não considerarei este modelo como viável.

**Rede Neural**

In [ ]:
# ---- Construção do modelo ----

# 1. Calcula o peso das classes
# Isso dirá à rede neural para prestar mais atenção à classe minoritária (AVC)
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {0: weights[0], 1: weights[1]}

print(f"Pesos das classes: {class_weights}")

In [ ]:
# 2. Definir a arquitetura do modelo
model = keras.Sequential([
    # Camada de entrada - o shape deve ser o número de features do seu X
    keras.Input(shape=(X_train_scaled.shape[1],)),

    # Primeira camada oculta - 32 neurônios, com uma função de ativação comum 'relu'
    keras.layers.Dense(32, activation='relu'),

    # Camada de Dropout para combater overfitting (boa prática)
    keras.layers.Dropout(0.3),

    # Segunda camada oculta
    keras.layers.Dense(16, activation='relu'),

    # Camada de saída - 1 neurônio com ativação 'sigmoid' para classificação binária
    # O sigmoid comprime a saída para um valor entre 0 e 1 (uma probabilidade)
    keras.layers.Dense(1, activation='sigmoid')
])

# Mostra um resumo da arquitetura criada
model.summary()

In [ ]:
# 3. Compila o modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

In [ ]:
# Callback para parar o treino se não houver melhora, evitando overfitting
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_recall', # Monitorar o recall na validação
    patience=10,          # Número de épocas sem melhora antes de parar
    restore_best_weights=True # Restaura os pesos da melhor época
)

# 4. Treina o modelo
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100, # Número máximo de épocas
    batch_size=32,
    validation_split=0.2, # Usa 20% dos dados de treino para validação a cada época
    class_weight=class_weights,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# ----- Teste da rede neural ----

# 1. Obtem as probabilidades no conjunto de teste
y_proba_nn = model.predict(X_test_scaled)

# 2. Aplica um threshold padrão de 0.5 para uma avaliação inicial
y_pred_nn_default = (y_proba_nn > 0.5).astype(int)

print("\n--- Relatório da Rede Neural com Threshold Padrão (0.5) ---")
print(classification_report(y_test, y_pred_nn_default))

# 3. Encontra e aplica o melhor threshold
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba_nn)

f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
f1_scores = f1_scores[:-1]
best_f1_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_f1_idx]

y_pred_nn_best = (y_proba_nn > best_threshold).astype(int)

print(f"\nMelhor Threshold encontrado (maximizando F1): {best_threshold:.4f}")
print(f"\n--- Relatório da Rede Neural com Threshold Otimizado ---")
print(classification_report(y_test, y_pred_nn_best))

Como uma última alternativa para tentar encontrar um modelo que consiga maior precisão sem reduzir tanto o recall, tentei aplicar uma rede neural simples para avaliar o resultado.

Mais uma vez consegui uma melhora no recall, mas a precisão caiu.

#Reavaliação da variável alvo

O resultado muito abaixo do esperado me fez voltar na análise exploratória, realizada na sprint anterior, e notei, através do gráfico abaixo que a maior concentração de casos de ocorrência de AVC está em pessoas com 60 anos ou mais.

In [ ]:
# Confirguração e título
fig, axes = plt.subplots(1, 1, figsize=(6, 6))
plt.suptitle('Idade vs. Ocorrência de AVC', fontsize=16)

# Boxplot 1: age (idade)
sns.boxplot(ax=axes, hue='stroke', x='stroke', y='age', data=avcpreproc, palette={0: 'lightblue', 1: 'red'}, legend=False)
axes.set_xlabel('')
axes.set_ylabel('Idade')
axes.set_xticks([0, 1])
axes.set_xticklabels(['Não teve AVC', 'Teve AVC'])

Decidi então aplicar um corte no dataset, mantendo somente os pacientes com 60 anos ou mais. Para não alongar muito mais o projeto, passarei por algumas etapas novamente de forma mais rápida, aplicando um pipeline final com os melhores resultados que encontrei.

O objetivo desse corte é deixar o modelo focar no grupo de risco e reduzir o desbalanceamento. De 1,80%, a proporção da classe minoritária passou para 5,32%, que poderá ser verificado abaixo.

Experimentei também uma abordagem diferente para a padronização, e cheguei à conclusão que padronizar somente as variáveis ***age*** e ***avg_glucose_level*** me permitiram um resultado levemente superior.

Ainda com o objetivo de não tornar o projeto muito maior do que se encontra, resumo através desse texto que realizei todos os testes novamente, com todas as técnicas de balanceamento vistas até aqui, com e sem validação cruzada. Nessa fase, o resultado mais interessante foi o NB utilizando UnderSample que atingiu precisão de 0.08 e recall de 0.75 com a validação cruzada com 5 folds.

Também testei novamente o desempenho da mesma rede neural com esse dataset filtrado, o resultado foi semelhante.

Mas o melhor resultado foi obtido através do uso do ensemble Voting com NB, Regressão Logística e Random Forest utilizando o dataset balanceado com UnderSample. Esse modelo será detalhado nas células seguintes.

**- Filtrando e exibindo o novo dataset**

In [ ]:
# Filtra o DataFrame para manter as linhas onde a idade é maior ou igual a 60
avcpreproc_filtered = avcpreproc[avcpreproc['age'] >= 60]

In [ ]:
# Verifica o dataset filtrado
avcpreproc_filtered.head()

In [ ]:
# Verificando as dimensões do dataset filtrado
print(f"Dimensões do Dataset: {avcpreproc_filtered.shape}")
print("-" * 30)

O dataset reduziu consideralvemente. Agora possui 32.262 linhas a menos.

As linhas restantes podem ajudar o modelo a diferenciar melhor o grupo de risco uma vez que todos os ruídos gerados pelas informações dos pacientes com menos de 60 anos foram removidos.

**- Separação entre treino e teste no dataset filtrado**

In [ ]:
# Separação dos Dados (Features e Target)
X = avcpreproc_filtered.drop('stroke', axis=1)
y = avcpreproc_filtered['stroke']

# Divisão estratificada em conjuntos de treino e teste para manter a proporção
X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered = train_test_split(X, y ,test_size=0.3, random_state=SEED, stratify = y)

print('Tamanho das separações:')
print(f"Treino: {X_train_filtered.shape}, {y_train_filtered.shape}")
print(f"Teste: {X_test_filtered.shape}, {y_test_filtered.shape}")
print(f"Proporção de stroke no treino: {y_train_filtered.mean():.4f}")
print(f"Proporção de stroke no teste: {y_test_filtered.mean():.4f}")

##Modelagem e otimização de hiperparâmetros

Na tentativa abaixo, foi utilizado o ensemble Voting com NB, Regressão Logística e Random Forest, como última tentativa da melhorar os resultados.



In [ ]:
# ---- Avaliação com Pipeline ----

# Componente A: O Pré-processador
# Define quais colunas serão padronizadas
vcontinuas = ['age', 'avg_glucose_level'] # Dessa vez a padronização não ocorrerá na coluna bmi
preprocessor = ColumnTransformer(transformers=[('scaler_continuas', StandardScaler(), vcontinuas)],remainder='passthrough')

# Componente B: O Balanceador
sampler = RandomUnderSampler(random_state=SEED) # Melhor balanceador encontrado

# Componente C: ensemble Voting como estimador base
clf_nb = GaussianNB()
clf_lr = LogisticRegression(random_state=SEED, solver='liblinear', max_iter=1000)
clf_rf = RandomForestClassifier(random_state=SEED)

# Cria o ensemble Voting
voting_model = VotingClassifier(estimators=[('nb', clf_nb), ('lr', clf_lr), ('rf', clf_rf)],voting='soft')

# --- Montagem do Pipeline ---
pipeline_voting = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampler', sampler),
    ('model', voting_model)
])

# Treinar e avaliar
pipeline_voting.fit(X_train_filtered, y_train_filtered)
y_pred = pipeline_voting.predict(X_test_filtered)
print(classification_report(y_test_filtered, y_pred))

Como podemos ver, finalmente conseguimos uma melhora na precisão, com o valor atingido não sendo visto em nenhum teste anterior, além disso, o recall manteve um patamar mínimo aceitável.

### Otimização dos hiperparâmetros utilizando GridSearch

In [ ]:
# 1. Padroniza
# Aplica o pré-processador
#X_train_scaled_filtered = preprocessor.fit_transform(X_train_filtered)
# Aplica a transformação no conjunto de teste
#X_test_scaled_filtered = preprocessor.transform(X_test_filtered)

#2. Aplica o balanceamento
# Balanceamento de Classes usando UnderSampling no conjunto de treino somente
#X_train_undersampled_filtered, y_train_undersampled_filtered = sampler.fit_resample(X_train_scaled_filtered, y_train_filtered)

# 3. Define o Grid de Hiperparametros
#param_grid = {
    #'lr__C': [0.1, 1.0, 10.0],  # Testa diferentes valores de regularização para a Regressão Logística
    #'lr__penalty': ['l1', 'l2'],
    #'lr__solver': ['liblinear', 'saga'],
    #'rf__n_estimators': [50, 100, 200], # Testa diferentes números de árvores para o Random Forest
    #'rf__max_depth': [None, 10, 20],   # Testa diferentes profundidades de árvore
    #'weights': [[1, 1, 1], [1, 2, 1], [1, 1, 2]] # Testa diferentes pesos de voto para os modelos
#}
# GaussianNB não tem hiperparâmetros importantes para serem otimizados, por isso o deixei de fora.

# 4. Configura o GridSearch
#    A métrica de otimização será o 'f1' para buscar um bom equilíbrio
#cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED) # Usei 5 splits para ser mais rápido e porque 10 não trouxe melhorias em outro teste

#grid_search = GridSearchCV(
    #estimator=voting_model, # O estimador foi definido na célula anterior
    #param_grid=param_grid,
    #scoring='f1', # Otimiza para o F1-Score
    #cv=cv_strategy,
    #n_jobs=-1,
    #verbose=2
#)

#print("Iniciando a otimização de hiperparâmetros para o VotingClassifier...")
# 5. Executa a busca
#    Treina com os dados de treino com UnderSample
#grid_search.fit(X_train_undersampled_filtered, y_train_undersampled_filtered)

# 6. Analie dos resultados
#print("\nMelhores hiperparâmetros encontrados:")
#print(grid_search.best_params_)

#print(f"\nMelhor score F1 (ponderado) da validação cruzada: {grid_search.best_score_:.4f}")

# 7. Avalia o melhor modelo no conjunto de teste
#print("\nAvaliação do melhor modelo encontrado no conjunto de teste:")
#best_voting_model = grid_search.best_estimator_
#y_pred = best_voting_model.predict(X_test_scaled_filtered)
#print(classification_report(y_test_filtered, y_pred, zero_division=0))

Devido ao tempo para execução da célula, superior a 7 minutos, optei por deixar o código comentado, mas deixando o resultado abaixo.

Melhores hiperparâmetros encontrados:
`{'lr__C': 0.1, 'lr__penalty': 'l1', 'lr__solver': 'saga', 'rf__max_depth': 10, 'rf__n_estimators': 50, 'weights': [1, 2, 1]}`

Métricas encontradas:

| Target | Precision | Recall | F1-score | Support |
| :--- | :--- | :--- | :--- | :--- |
| 0 | 0.98 | 0.59 | 0.74 | 3161 |
| 1 | 0.10 | 0.77 | 0.17 | 178 |


###Pipeline e teste final

**- Pipeline final**

In [ ]:
# Componente A: O Pré-processador
# Define quais colunas serão padronizadas
vcontinuas = ['age', 'avg_glucose_level'] # Dessa vez a padronização não ocorrerá na coluna bmi
preprocessor = ColumnTransformer(transformers=[('scaler_continuas', StandardScaler(), vcontinuas)],remainder='passthrough')

# Componente B: O Balanceador
sampler = RandomUnderSampler(random_state=SEED) # Melhor balanceador encontrado

# Componente C: O Modelo
# Modelo base com hiperparâmetros ajustados
clf_nb = GaussianNB()
clf_lr = LogisticRegression(random_state=SEED,solver='saga', penalty='l1', C=0.1, max_iter=1000)
clf_rf = RandomForestClassifier(random_state=SEED,n_estimators=50, max_depth=10)

# Cria o ensemble Voting com hiperparâmetros ajustados
best_model = VotingClassifier(estimators=[('nb', clf_nb), ('lr', clf_lr), ('rf', clf_rf)], voting='soft', weights=[1, 2, 1])

# --- Montagem do Pipeline Final ---
final_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampler', sampler),
    ('model', best_model)
])

O pipeline final foi definido através de intensos testes. Com o dataset filtrado e padronizado dessa forma, o balancemaneto com UnderSample foi o que trouxe melhores resultados.

**- Teste final**

In [ ]:
# Teste e avaliação finais
final_pipeline.fit(X_train_filtered, y_train_filtered)
y_pred_best = final_pipeline.predict(X_test_filtered)
print(classification_report(y_test_filtered, y_pred_best))

Após exaustivos testes, utilizando diversos modelos e configurações, o modelo que obteve melhor desempenho foi o ensemble Voting utilizando o dataset padronizado e com UnderSample, devido ao recall mais elevado na classe onde ocorre o AVC e à precisão que foi mais alta. Isso aliado ao tempo de treinamento indica ser o melhor modelo para seguir no momento.


# Avaliação final, análise de erros e limitações

**- Melhor modelo vs Baseline**

In [ ]:
print('Resultado da baseline:')
print(classification_report(y_test, y_pred_baseline, zero_division=0))

print('Resultado do melhor modelo:')
print(classification_report(y_test_filtered, y_pred_best))

A análise comparativa demonstra um avanço significativo e indiscutível do "Melhor Modelo" sobre a baseline, pois ele foi capaz de aprender a identificar a classe minoritária, alcançando um recall de 77% para os casos de AVC, uma tarefa em que a baseline falhou completamente com 0%.

Isso é um salto gigantesco e a prova de que o modelo aprendeu um padrão preditivo valioso.

Essa conquista representa a transição de um modelo inútil para um com valor prático, que estabelece um sinal preditivo real, ainda que a baixa precisão de 10% mostre a necessidade de refinamento. O custo para esse ganho foi uma queda esperada na performance da classe majoritária, confirmando que o modelo está fazendo um trade-off ativo, e o desafio agora é aprimorar sua confiabilidade, melhorando a precisão sem sacrificar significativamente o recall já obtido.

**- Matriz de confusão e análise de erros**

In [ ]:
# Cria a matriz de confusão
cm = confusion_matrix(y_test_filtered, y_pred_best)
print("\nMatriz de confusão:")
print(cm)

# Cria o gráfico com Seaborn
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')

# Adiciona rótulos
plt.xlabel('Rótulo Previsto')
plt.ylabel('Rótulo Verdadeiro')

# Mostra o gráfico
plt.show()

A análise da matriz de confusão revela um modelo que foi bem-sucedido em sua principal tarefa de detecção, alcançando um recall de 77%, o que significa que ele identificou corretamente a maioria dos casos reais de AVC e minimizou o erro mais crítico (Falsos Negativos). No entanto, essa alta sensibilidade teve um custo significativo na confiabilidade, evidenciado pela baixíssima precisão, resultado de um número excessivo de 1300 alarmes falsos (Falsos Positivos). Isso caracteriza o modelo como um detector "sensível, mas pouco confiável", cujo desafio agora é refinar sua capacidade de discernimento para aumentar a precisão sem comprometer drasticamente sua capacidade de detecção.

**- Limitações**

A característica mais dominante e definidora do projeto é que os dados são extremamente desbalanceados, com a classe de AVC sendo muito rara. Mais importante ainda, a extensa experimentação provou que os atributos disponíveis, embora relevantes, possuem um sinal preditivo fraco. Isso significa que não há uma combinação clara e forte de características que separe inequivocamente os pacientes que terão um AVC dos que não terão. A informação está "diluída" em muito ruído, tornando a tarefa de classificação inerentemente difícil.

Rapidamente notei que a acurácia é uma métrica perigosa e enganosa para este problema, pois uma baseline que sempre prevê "não AVC" alcança uma acurácia altíssima, mas é completamente inútil. A verdadeira história do projeto foi contada pelo conflito direto entre a Precisão e o Recall da classe 1 (AVC). Todas as tentativas de aumentar o Recall, seja com balanceamento ou modelos complexos, resultaram em uma queda drástica da Precisão. As métricas, portanto, não apenas mediram a performance, mas também serviram como uma ferramenta de diagnóstico, revelando consistentemente que o modelo não conseguia ser sensível e confiável ao mesmo tempo com os dados atuais.

O projeto começou com um modelo de baseline que tinha um viés massivo para a classe majoritária, aprendendo a ignorar completamente a classe minoritária. Todas as técnicas subsequentes (SMOTE, UnderSampling, class_weight) foram esforços para combater esse viés e forçar os modelos a prestar atenção nos casos de AVC. Embora isso tenha sido bem-sucedido em aumentar o Recall, introduziu um novo padrão de erro: uma tendência a gerar Falsos Positivos. Essencialmente, trocamos um viés de "omissão" por um viés de "excesso de alerta", um passo necessário, mas que evidencia a dificuldade de encontrar um ponto de operação verdadeiramente sem viés.

A conclusão é que, mesmo com as melhores práticas, a fraca capacidade de generalização dos modelos está diretamente ligada à fraqueza do sinal preditivo nos dados de origem.

# Conclusões e próximos passos

As hipóteses iniciais foram postas à prova e o projeto foi bem-sucedido em desenvolver um modelo significativamente superior à baseline, com a capacidade de detectar a maioria dos casos de AVC ao atingir um alto recall após a aplicação de diversas técnicas. Contudo, essa sensibilidade foi alcançada ao custo de uma baixa precisão, um trade-off imposto pela natureza desbalanceada e pelo sinal preditivo fraco dos dados, onde mesmo algoritmos complexos e otimizados não conseguiram generalizar de forma eficaz. A conclusão definitiva é que o projeto atingiu um teto de performance imposto pelos atributos atuais, e as melhorias futuras mais promissoras não virão de novos modelos, mas sim de um foco estratégico na aquisição de novos dados para a classe minoritária, na incorporação de novas variáveis de entrada e na engenharia de atributos para enriquecer a informação disponível e, consequentemente, aprimorar a precisão do modelo sem sacrificar drasticamente sua já robusta capacidade de detecção.